In [1]:
from WikiXmlHandler import WikiXmlHandler
import xml.sax
import mwparserfromhell

import subprocess
import pandas as pd
import os

In [2]:
%%time
directory = "./data/"
files = os.listdir(directory)

handler = WikiXmlHandler("./saved/out_d.csv", "./saved/infobox-list.csv")
handler._DEBUG = True

parser = xml.sax.make_parser()
parser.setContentHandler(handler)

# for file in files:
for line in subprocess.Popen(["bzcat"],
                        stdin=open(directory + files[0]),
                        stdout=subprocess.PIPE).stdout:
    
    if handler._counter > 50:
        break

    parser.feed(line)
    # making sure that everything was written to the file
# handler.write_df()

Wall time: 5.08 s


In [6]:
n_df = pd.read_csv("./saved/out.csv", delimiter="\t")
n_df[n_df.title.str.contains("(?i)cobol")]

,title,type,def words,infobox_name,infobox,categories,wikilinks
3037,CoBoL,redirect,NaN,NaN,NaN,NaN,COBOL
4193,COBOL,article,NaN,NaN,NaN,COBOL|.NET programming languages|1959 software...,Kobol|File:COBOL Report Apr60.djvu|CODASYL|Pro...


In [3]:
for i, a in enumerate(handler._data):
    # if a[0] == "COBOL":
    print("{}: {}".format(i, a[0]))

0: Anarchism
1: Autism
2: Albedo
3: A
4: Alabama
5: Achilles
6: Abraham Lincoln
7: Aristotle
8: An American in Paris
9: Academy Award for Best Production Design
10: Academy Awards
11: Actrius
12: Animalia (book)
13: International Atomic Time
14: Altruism
15: Ayn Rand
16: Alain Connes
17: Allan Dwan
18: Algeria
19: List of Atlas Shrugged characters
20: Anthropology
21: Agricultural science
22: Alchemy
23: Alien
24: Astronomer
25: ASCII
26: Austin (disambiguation)
27: Animation
28: Apollo
29: Andre Agassi
30: Austroasiatic languages
31: Afroasiatic languages
32: Andorra
33: Arithmetic mean
34: American Football Conference
35: Animal Farm
36: Amphibian
37: Alaska
38: Agriculture
39: Aldous Huxley
40: Ada
41: Aberdeen (disambiguation)
42: Algae
43: Analysis of variance
44: Alkane
45: Appellate procedure in the United States
46: Answer (law)
47: Appellate court
48: Arraignment
49: America the Beautiful
50: Assistive technology


In [3]:
page = handler._data[6]

In [5]:
page[4].split("|")

['name:Abraham Lincoln',
 'image:Abraham Lincoln O-77 matte collodion print.jpg',
 'caption:Lincoln in November 1863',
 'alt:An iconic photograph of a bearded Abraham Lincoln showing his head and shoulders.',
 'office:16th President of the United States',
 'vicepresident:Hannibal Hamlin  (1861–1865)  Andrew Johnson  (March–April 1865)',
 'term_start:March 4, 1861',
 'term_end:April 15, 1865',
 'predecessor:James Buchanan',
 'successor:Andrew Johnson',
 'state1:Illinois',
 'district1:IL 7 7th',
 'term_start1:March 4, 1847',
 'term_end1:March 3, 1849',
 'predecessor1:John Henry',
 'successor1:Thomas L. Harris',
 'office2:Member of the  Illinois House of Representatives  from Sangamon County',
 'term_start2:December 1, 1834',
 'term_end2:December 4, 1842',
 'birth_date:1809 2 12',
 'birth_place:Sinking Spring Farm, Kentucky, U.S.',
 'death_date:1865 4 15 1809 2 12',
 'death_place:Washington, D.C., U.S.',
 'death_cause:Assassination (gunshot wound to the head)',
 'restingplace:Lincoln Tomb

In [3]:
infoboxes = pd.read_csv("../saved/infobox-list.csv")["infobox template name"].values.tolist()
cols = ["title", "type", "def words", "infobox_name", "infobox", "categories","wikilinks"]

In [4]:
# Creating dict
d = {col: [] for col in cols}
df = pd.DataFrame(columns=cols)
# parse page
# wiki = mwparserfromhell.parse(page[1])

In [8]:
t = [None, "3", "None", "6", None, None,"1"]

In [9]:
[d[cols[i]].append(t[i]) for i in range(len(t))]

[None, None, None, None, None, None, None]

In [10]:
d

{'title': [None, None],
 'type': ['3', '3'],
 'def words': [None, 'None'],
 'infobox_name': ['6', '6'],
 'infobox': ['8', None],
 'categories': [None, None],
 'wikilinks': ['1', '1']}

In [256]:
df

,title,type,def words,infobox_name,infobox,categories,wikilinks


In [238]:
d

{'title': [],
 'type': [],
 'def words': [],
 'infobox_name': [],
 'infobox': [],
 'categories': [],
 'wikilinks': []}

In [257]:
# page title
# d["title"].append(page[0])
df["title"] = [page[0]]

In [258]:
# defining article type
if page[2] != None:
    # d["type"].append("redirect")
    df["type"] = ["redirect"]
else:
    # d["type"].append("article")
    df["type"] = ["article"]

In [259]:
# Getting article categories
# d["categories"].append(
#     "|".join([
#         list(filter(None, x.title.split(":")))[-1]
#         for x in wiki.filter_wikilinks(matches="category")
#     ]))
df["categories"] = ["|".join([
        list(filter(None, x.title.split(":")))[-1]
        for x in wiki.filter_wikilinks(matches="category")
    ])]

In [260]:
# wikilinks, removing links to Category items
# d["wikilinks"].append("|".join(
#     [link.title.strip_code().strip() for link in wiki.filter_wikilinks() if "Category:" not in link.title.strip_code().strip()]))

df["wikilinks"] = ["|".join(
    [link.title.strip_code().strip() for link in wiki.filter_wikilinks() if "Category:" not in link.title.strip_code().strip()])]

# external links
# df["ext_links"] = "|".join([link.url.strip_code().strip() for link in wiki.filter_external_links()])

In [261]:
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [262]:
# search for infobox template name
box = None
for template in wiki.filter_templates():
    if template.name.strip() in infoboxes:
        box = wiki.filter_templates(matches=template.name.strip())[0]
        # d["infobox_name"].append(template.name.strip())
        df["infobox_name"] = [template.name.strip()]
        break

a = "|".join(["{}:{}".format(param.name.strip_code().strip(),
                            param.value.strip_code(keep_template_params=True).strip()) for param in box.params])
# d["infobox"].append(cleanhtml(a))
df["infobox"] = [cleanhtml(a)]

In [263]:
df

,title,type,def words,infobox_name,infobox,categories,wikilinks
0,COBOL,article,NaN,Infobox programming language,name:COBOL|logo:200px|alt=COBOL. Report to Con...,COBOL|.NET programming languages|1959 software...,Kobol|File:COBOL Report Apr60.djvu|CODASYL|Pro...


In [264]:
df["infobox"][0].split("|")

['name:COBOL',
 'logo:200px',
 'alt=COBOL. Report to Conference on Data Systems Languages including initial specifications for a Common Business Oriented Language (COBOL) for programming digital electronic computers. Department of Defense, April 1960.',
 'logo caption:The cover of the COBOL 60 report to CODASYL (April 1960)',
 'paradigm:Procedural, imperative, object-oriented',
 'year:yes 1959',
 'designers:Howard Bromberg, Howard Discount, Vernon Reeves, Jean E. Sammet, William Selden, Gertrude Tierney, Mary K. Hawes',
 'developers:CODASYL, ANSI, ISO',
 'latest release version:ISO/IEC 1989:2014',
 'latest release date:2014',
 'typing:Weak, static',
 'implementations:GnuCOBOL, IBM COBOL, Micro Focus Visual COBOL',
 'dialects:ACUCOBOL-GT, COBOL-IT, COBOL/2, DEC COBOL-10, DEC VAX COBOL, DOSVS COBOL, Fujitsu COBOL, Hitachi COBOL2002, HP3000 COBOL/II, IBM COBOL SAA, IBM COBOL/400, IBM COBOL/II, IBM Enterprise COBOL, IBM ILE COBOL, IBM OS/VS COBOL, ICL COBOL (VME),  isCOBOL, Micro Focus COB

In [265]:
df["wikilinks"][0].split("|")

['Kobol',
 'File:COBOL Report Apr60.djvu',
 'CODASYL',
 'Procedural programming',
 'imperative programming',
 'Object-oriented programming',
 'Jean E. Sammet',
 'CODASYL',
 'ANSI',
 'ISO',
 'weak typing',
 'static typing',
 'GnuCOBOL',
 'IBM COBOL',
 'Micro Focus',
 'AIMACO',
 'C++',
 'COMTRAN',
 'Eiffel (programming language)',
 'FACT computer language',
 'FLOW-MATIC',
 'Smalltalk',
 'CobolScript',
 'EGL (programming language)',
 'PL/I',
 'Academic Press',
 'Programming Language for Business',
 'acronym',
 'WP:EXPABBR',
 'compiled',
 'computer programming language',
 'imperative programming',
 'procedural programming',
 'object-oriented programming',
 'mainframe computer',
 'batch processing',
 'transaction processing',
 'CODASYL',
 'Grace Hopper',
 'Basic Books',
 'US Department of Defense',
 'Software portability',
 'MIT Press',
 'standardized',
 'structured programming',
 'object-oriented programming',
 'ISO',
 'International Electrotechnical Commission',
 'reserved word',
 'standa

In [223]:
# mais proximo da solucao definitiva
for param in box.params:
    for x in param.value.filter_templates():
        print([y.value.strip_code() for y in x.params])

['yes', '1959']
['Howard Bromberg']
['Howard Discount']
['Vernon Reeves']
['Jean E. Sammet']
['William Selden']
['Gertrude Tierney']
['Mary K. Hawes']
['GnuCOBOL, IBM COBOL']
['Micro Focus Visual COBOL']
['2002 influence', ' " SW95 " / > < ref name= " Arranga98 " / > < ref >  < /ref > ']
['Cobol: Perception and Reality ', 'Computer ', 'March 1997 ', '30 ', '3 ', '10.1109/2.573683 ', '127 ', '0018-9162 ', 'Edmund C. ', 'Arranga ', 'Frank P. ', 'Coyle ']
['2002 influence']
['2002 influence']
['COBOL Script: a business-oriented scripting language ', 'Tetsuji ', 'Imajo ', 'Tatsuki ', 'Miyake ', 'Shinobu ', 'Sato ', 'Toshiyuki ', 'Ito ', 'Daisuke ', 'Yokotsuka ', 'Yoshihide ', 'Tsujihata ', 'Shunsuke ', 'Uemura ', '1 ', 'IEEE ', 'September 2000 ', 'Enterprise Distributed Object Computing Conference ', 'http://dblp1.uni-trier.de/db/conf/edoc/edoc2000.html ', 'Makuhari, Japan ', '0769508650 ', '10.1109/EDOC.2000.882363 ']
['https://www.omg.org/adm/EGLOverviewtoOMG.pdf ', 'Introduction to EGL 

In [135]:
# box.params.strip_code()
for param in box.params:
    # print(param.value.filter_text())
    # print(param.value.filter_wikilinks())
    print(param.value.strip_code(keep_template_params=True).strip())

COBOL
200px|alt=COBOL. Report to Conference on Data Systems Languages including initial specifications for a Common Business Oriented Language (COBOL) for programming digital electronic computers. Department of Defense, April 1960.
The cover of the COBOL 60 report to CODASYL (April 1960)
Procedural, imperative, object-oriented
yes 1959
Howard Bromberg, Howard Discount, Vernon Reeves, Jean E. Sammet, William Selden, Gertrude Tierney, Mary K. Hawes
CODASYL, ANSI, ISO
ISO/IEC 1989:2014
2014
Weak, static
GnuCOBOL, IBM COBOL, Micro Focus Visual COBOL
ACUCOBOL-GT, COBOL-IT, COBOL/2, DEC COBOL-10, DEC VAX COBOL, DOSVS COBOL, Fujitsu COBOL, Hitachi COBOL2002, HP3000 COBOL/II, IBM COBOL SAA, IBM COBOL/400, IBM COBOL/II, IBM Enterprise COBOL, IBM ILE COBOL, IBM OS/VS COBOL, ICL COBOL (VME),  isCOBOL, Micro Focus COBOL, Microsoft COBOL, Realia COBOL, Ryan McFarland RM/COBOL, Ryan McFarland RM/COBOL-85, Tandem (NonStop) COBOL85, Tandem (NonStop) SCOBOL, UNIVAC COBOL, Unisys MCP COBOL74, Unisys MCP

In [46]:
df["infobox"][0]

'|name:COBOL||logo:200px|alt=COBOL. Report to Conference on Data Systems Languages including initial specifications for a Common Business Oriented Language (COBOL) for programming digital electronic computers. Department of Defense, April 1960.||logo caption:The cover of the COBOL 60 report to CODASYL (April 1960)||paradigm:Procedural, imperative, object-oriented||year:||designers:, , , , , ,||developers:CODASYL, ANSI, ISO||latest release version:ISO/IEC 1989:2014||latest release date:2014||typing:Weak, static||implementations:,||dialects:ACUCOBOL-GT, COBOL-IT, COBOL/2, DEC COBOL-10, DEC VAX COBOL, DOSVS COBOL, Fujitsu COBOL, Hitachi COBOL2002, HP3000 COBOL/II, IBM COBOL SAA, IBM COBOL/400, IBM COBOL/II, IBM Enterprise COBOL, IBM ILE COBOL, IBM OS/VS COBOL, ICL COBOL (VME),  isCOBOL, Micro Focus COBOL, Microsoft COBOL, Realia COBOL, Ryan McFarland RM/COBOL, Ryan McFarland RM/COBOL-85, Tandem (NonStop) COBOL85, Tandem (NonStop) SCOBOL, UNIVAC COBOL, Unisys MCP COBOL74, Unisys MCP COBOL8